In [2]:
print ("This is the start of the notebook")
print ("Hopefully we can learn a thing or two, before doing a thing or two.")

This is the start of the notebook
Hopefully we can learn a thing or two, before doing a thing or two.


In [6]:
from elasticsearch import Elasticsearch
import json

endpoint = "https://5a5b8a5cdd84464dae4c7c7ae8a59562.us-east1.gcp.elastic-cloud.com:443"
api_key = "aTN4MUdwVUJLTFFTSmFFWjBlTFM6dmU0ZXJnTjdUaUs5dXhIUU1fd0xiZw=="

es_client = Elasticsearch(
    endpoint,
    api_key=api_key,
)

In [7]:
index_name = "search-migration_crawler"
if not es_client.indices.exists(index=index_name):
    print ("Eek! The index does not exist!")
else:
    print (f"The index [{index_name}] was found!")

The index [search-migration_crawler] was found!


In [8]:
print ("Let's see if we can get _all_ indices in this ES instance related to crawler?\n")

json_response = es_client.cat.indices(
    index=".*crawler*", # take note of the . before *crawler* - this tells the API to query 'hidden' indices as well
    s="index",
    format="json"
)

print (f"We got the list as a JSON dictionary! We received {len(json_response)} indices.\n")

health_histogram = {
    "green": 0,
    "yellow": 0,
    "red": 0,
}

indices_with_docs = {
    "with_docs": 0,
    "without_docs": 0,
}

index_names = [] # save the index names to run through all of them at some point in the future

for item in json_response.body: # Note that calling .body on the response will get us a List of dictionaries:
    health_status = item["health"]
    health_histogram[health_status] += 1

    if int(item['docs.count']) > 0:
        indices_with_docs['with_docs'] += 1
        index_names.append(item['index'])
    else:
        indices_with_docs['without_docs'] += 1

    print (f"    {item['docs.count']} docs -> {item['index']}")

print (f"\nThere are {health_histogram['green']} healthy indices, {health_histogram['yellow']} sick indices \
and {health_histogram['red']} unhealthy indices.")

print (f"{indices_with_docs['with_docs']} indices have docs, and {indices_with_docs['without_docs']} \
indices do not.")

Let's see if we can get _all_ indices in this ES instance related to crawler?

We got the list as a JSON dictionary! We received 20 indices.

    4986743 docs -> .ds-logs-elastic_crawler-default-2025.02.05-000001
    0 docs -> .ent-search-actastic-app_search_crawler_content_metadata
    0 docs -> .ent-search-actastic-app_search_crawler_content_metadata-content_hash-engine_oid-unique-constraint
    0 docs -> .ent-search-actastic-app_search_crawler_content_url_metadata
    3 docs -> .ent-search-actastic-crawler2_configurations_v2
    3 docs -> .ent-search-actastic-crawler2_configurations_v2-index_name-unique-constraint
    9385 docs -> .ent-search-actastic-crawler2_content_metadata
    9385 docs -> .ent-search-actastic-crawler2_content_metadata-configuration_oid-content_hash-unique-constraint
    9532 docs -> .ent-search-actastic-crawler2_content_url_metadata
    332 docs -> .ent-search-actastic-crawler2_crawl_requests_v2
    4 docs -> .ent-search-actastic-crawler2_domains
    4 docs -> 

In [32]:
print ("Let's grab the extraction rules for our crawler.")
print ("The extraction rules are defined in the index .ent-search-actastic-crawler2_extraction_rules\n")

ex_r = es_client.search(
    index=".ent-search-actastic-crawler2_extraction_rules",
    # _source="rules"
)
import pprint as pp

pp.pprint (dict(ex_r["hits"]))
print ()

for i in ex_r["hits"]["hits"]:
    print (f"Index: {i['_index']}")
    rules = i["_source"]["rules"]
    print (rules)
    for rule in rules:
        print (f"Description: {i['_source']['description']}")
        print (f"Configuration OID: {i['_source']['configuration_oid']}") # <-- I wonder if we can use these to
        print (f"Domain OID: {i['_source']['domain_oid']}")               # <-- match specific crawlers
        
        print (f"Value type: {rule['content_from']['value_type']}") # <-- this maps to 'action'
        print (f"Value: {rule['content_from']['value']}") # <--
        print (f"Field name: {rule['field_name']}") # <--
        print (f"Multiple Objects Handling: {rule['multiple_objects_handling']}") # <-- this maps to "join_as"
        print (f"Selector: {rule['selector']}") # <--
        print (f"Source type: {rule['source_type']}\n") # <--

Let's grab the extraction rules for our crawler.
The extraction rules are defined in the index .ent-search-actastic-crawler2_extraction_rules

{'hits': [{'_id': '67a393cc17490cc0d24853ef',
           '_index': '.ent-search-actastic-crawler2_extraction_rules',
           '_score': 1.0,
           '_source': {'configuration_oid': '67a3857117490c2adf48274d',
                       'created_at': '2025-02-05T16:37:32Z',
                       'description': 'extraction_rule_generic',
                       'domain_oid': '67a3858717490ccc74482755',
                       'edited_by': '1432693181',
                       'id': '67a393cc17490cc0d24853ef',
                       'rules': [{'content_from': {'value': '"some_rando_value"',
                                                   'value_type': 'fixed'},
                                  'field_name': 'test_field',
                                  'multiple_objects_handling': 'string',
                                  'selector': '/html

In [8]:
print ("Let's try grabbing the configurations docs in .ent-search-actastic-crawler2_configurations_v2")

config_r = es_client.search(
    index=".ent-search-actastic-crawler2_configurations_v2",
)

print (config_r)
print ()

print ("Inside this index, we can find the following values:\n")
for i in config_r["hits"]["hits"]:
    source = i["_source"]
    print (f"Index: {i['_index']}")
    print (f"Configuration ID: {source['id']}") # <--
    print (f"Index name: {source['index_name']}") # <--
    print (f"Crawl schedule: {source['crawl_schedule']}")
    print (f"Use crawl schedule?: {source['use_connector_schedule']}\n")

Let's try grabbing the configurations docs in .ent-search-actastic-crawler2_configurations_v2
{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': '.ent-search-actastic-crawler2_configurations_v2', '_id': '67a3857117490c2adf48274d', '_score': 1.0, '_source': {'id': '67a3857117490c2adf48274d', 'created_at': '2025-02-05T15:36:17Z', 'updated_at': '2025-02-05T16:34:04Z', 'index_name': 'search-migration_crawler', 'crawl_schedule': [{'unit': 'hour', 'frequency': 1}], 'use_connector_schedule': False}}, {'_index': '.ent-search-actastic-crawler2_configurations_v2', '_id': '67adf643cf03320a318aeefa', '_score': 1.0, '_source': {'id': '67adf643cf03320a318aeefa', 'created_at': '2025-02-13T13:40:19Z', 'updated_at': '2025-02-13T14:29:16Z', 'index_name': 'search-daggerfall-unity-website-crawler', 'crawl_schedule': [{'unit': 'hour', 'frequency': 24}], 'use_connector_sc

In [9]:
print (".ent-search-actastic-crawler2_domains\n")

domains_r = es_client.search(
    index=".ent-search-actastic-crawler2_domains",
    _source=["name",
             "configuration_oid",
             "id",
             "sitemaps",
             "crawl_rules",
             "seed_urls",
             "auth"]
)

print (domains_r)
print ()

for i in domains_r["hits"]["hits"]:
    source = i["_source"]
    print (f"Name: {source['name']}") # <--
    
    print (f"Configuration OID: {source['configuration_oid']}") # <--
    print (f"Domain ID: {source['id']}") # <--
        
    print (f"Sitemaps : {source['sitemaps']}") # <--
    print (f"Crawl rules: {source['crawl_rules']}") # <--
    print (f"Seed URLs: {source['seed_urls']}") # <--
    
    print (f"Auth (decode and split on the ':' delimiter): {source['auth']}\n") # <--

.ent-search-actastic-crawler2_domains

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': '.ent-search-actastic-crawler2_domains', '_id': '67abc186cf0332829183f93b', '_score': 1.0, '_source': {'id': '67abc186cf0332829183f93b', 'configuration_oid': '67abc13fcf0332544683f928', 'name': 'https://www.speedhunters.com', 'crawl_rules': [], 'seed_urls': [{'created_at': '2025-02-11T21:31:04Z', 'id': '67abc198cf0332d5ae83f93d', 'url': 'https://www.speedhunters.com/2025/01/the-mystery-of-the-hks-zero-r/'}], 'sitemaps': [{'created_at': '2025-02-14T16:58:47Z', 'id': '67af7647cf0332fdd191f64d', 'url': 'https://www.speedhunters.com/post_tag-sitemap2.xml'}], 'auth': 'akh4My83Myt3Y2NkbnJHUll2RFVlZz09LS1wbjU5TEZmNVdxUDZCK0lSa2dwRlhRPT0=--21413c9faa1309012319bdf4132ef5f4d0add9fb'}}, {'_index': '.ent-search-actastic-crawler2_domains', '_id': '67adf64bcf0332cb308aef02', 

In [8]:
all_domain_ids = []
all_config_ids = []
for i in domains_r["hits"]["hits"]:
    source = i["_source"]
    
    all_domain_ids.append(source['id'])

    if source['configuration_oid'] not in all_config_ids:
        all_config_ids.append(source['configuration_oid'])

print (all_domain_ids)
print (all_config_ids)

['67adf64bcf0332cb308aef02', '67abc186cf0332829183f93b', '67a3867317490c4bca482807', '67a3858717490ccc74482755']
['67adf643cf03320a318aeefa', '67abc13fcf0332544683f928', '67a3857117490c2adf48274d']


In [41]:
import yaml
import os
import pprint as pp

##### You may change the value of file_name #####
file_name = "my-output-conf.yml"
#################################################

data = {
        "url": "",
        "seed_urls": ["seed1", "seed2", "seed3"],
        "sitemap_urls": ["sitemap1", "sitemap2"],
        'extraction_rulesets':[{
                "url_filters": [{
                    "type": "begins",
                    "pattern": "/cool/pattern/*"
                }],
                "rules": [{
                    "action": "extract",
                    "field_name": "author",
                    "selector": ".author",
                    "join_as": "array",
                    "value": "yes",
                    "source": "html"
                }]
            }]
}
# pp.pprint(data)

base_dir = os.getcwd()

output_path = os.path.join(base_dir, file_name)
print (output_path)

if os.path.exists(base_dir):
    with open(output_path, 'w') as file:
        yaml.dump(data, file)

/Users/mattnowzari/repos/search_and_transform/crawler/migration/my-output-conf.yml
